# 🧪 Workshop 1: From Gradient Flow to Optimisation Intuition
> This notebook is provided in a clean, non-executed version for the reader to try out on the problem.
>
> A model answer and executive summary can be found in the *worked* version.

This set of workshops builds on Tutorials 1–4 and closes **Part 1** of the series, consolidating core ideas about PyTorch `autograd` and gradient flow before moving on to explicit optimisation in and Part 2.

Rather than treating gradients as black-box training signals, the workshop emphasises **gradients as sensitivity measures**, especially in settings where model outputs are **tensor-valued** and gradients are computed using **explicit upstream directions**.

---
**The problems are designed to shift perspective**:
- from *“what gradient does PyTorch give me?”*
- to *“what does this gradient represent, and why?”*
---
**The emphasis is on developing intuition for**:
- how gradients flow through structured linear and nonlinear computations,
- how upstream gradients select directions in output space,
- how `.grad` reflects sensitivity rather than optimisation,
- and how gradient structure anticipates optimisation behaviour.
---
**Key ideas explored include**:
- vector–Jacobian products as the fundamental object computed by `backward(v)`,
- the role of upstream gradients in shaping gradient flow,
- sparsity and structure in gradients induced by linear maps and nonlinearities (e.g., ReLU),
- interpreting gradients statistically and geometrically rather than procedurally,
- and using controlled experiments to reason about gradient behaviour.
---
**This workshop serves as a conceptual bridge between**:
- `autograd` mechanics and gradient flow (Tutorials 3–4),
- and **objective design and optimisation dynamics** (Workshop 2 and Part 2).

The focus is intentionally *not* on training pipelines, datasets, or optimisers, but on understanding how gradients behave in controlled settings—laying the groundwork for effective optimisation.

---
**Recommended prerequisites**:
- Familiarity with PyTorch tensors and `.backward()`
- Understanding of scalar vs tensor-valued gradients
- Basic comfort with linear algebra and nonlinear activations

---

**Author: Angze Li**

**Last updated: 2026-02-19**

**Version: v1.1**

## 🧩 Problem: Sensitivity of a Feature Transformation

>In many models, inputs are transformed into feature representations before being used by a downstream objective.
> Understanding which inputs influence which features is critical for interpretability and optimisation.

Consider the following setup:
```python
x = torch.randn(6, requires_grad=True)

W = torch.tensor([
    [1.0,  0.0,  0.0, 0.0, 0.0, 0.0],
    [0.0,  1.0,  0.0, 0.0, 0.0, 0.0],
    [0.0,  0.0,  2.0, 0.0, 0.0, 0.0],
    [0.0,  0.0,  0.0, 3.0, 0.0, 0.0],
], requires_grad=False)

out = torch.relu(W @ x) # what does this line do?
```
This produces a 4-dimensional tensor output, not a scalar.

---

### Task
1.	Construct an upstream gradient vector `v` such that:
    - the last feature of out is weighted most heavily,
    - the first two features are ignored.
2. Call:
```python
out.backward(v)
```
3. Inspect `x.grad`.

---

### Questions to think about
- Which components of `x` receive non-zero gradients?
- How does the structure of `W` affect gradient flow?
- How does ReLU change which inputs are “active”?
- Why is `x.grad` sparse in some cases?

---

### Hint

> Think in terms of which inputs influence which outputs, and which outputs are being emphasised by `v`.

---

### Learning outcomes

After this problem, you should be comfortable with:
- interpreting `.grad` as a sensitivity map,
- reasoning about gradient flow through linear + nonlinear layers,
- understanding how upstream weighting reshapes gradient influence.

This problem bridges Tutorials 3 and 4 cleanly and warms up the optimisation mindset.

---
**Author**: Angze Li

**Last updated**: 2026-02-20

**Version**: v2.0

## Solution

## Executive summary

### Understanding the solution

This exercise explores how gradients flow through a simple linear–nonlinear system when the output is tensor-valued and gradients are computed via a vector–Jacobian product.

The model we studied is:
$$\text{out} = \operatorname{ReLU}(W x),$$
with a user-chosen upstream gradient `v` supplied to `backward()`.


### Which components of `x` receive non-zero gradients?

Only a **subset of the components of `x`** ever receive non-zero gradients.

This happens for two reasons:
1. **Linear structure**: Each row of `W` depends only on one component of `x.

    As a result, each output entry is influenced by exactly one input coordinate.
2. **Upstream weighting**: Only the output components with non-zero entries in `v` contribute to the gradient.

If a component of `x` does not influence any weighted output entry, its gradient is exactly zero.

### How does the structure of `W` affect gradient flow?

The matrix `W` acts as a **routing mechanism** for gradients.

Because `W` is sparse and diagonal-like:
- each output dimension maps directly to a single input dimension,
- gradient flow is strictly localised,
- there is no mixing between unrelated components of `x`.

This makes the gradient behaviour easy to interpret:
changing the weight of one output only affects the gradient of its corresponding input.

### How does ReLU change which inputs are “active”?

ReLU introduces **input-dependent gating**.

For each output component:
$$
\operatorname{ReLU}(z) =
\begin{cases}
z, & z > 0 \\
0, & z \le 0
\end{cases}
$$

This means:
- gradients flow **only when the pre-activation is positive**,
- inputs corresponding to negative pre-activations are *temporarily invisible* to backpropagation.

In this exercise, averaging gradients over many random samples reveals that:
- each active ReLU contributes gradients roughly **half** the time,
- inactive paths contribute nothing.

This explains the appearance of **fractional average gradients** in the Monte Carlo results.

### Why is `x.grad` sparse in some cases?

`x.grad` is sparse because gradient flow is conditional:
- conditional on which outputs are weighted by `v`,
- conditional on which ReLU units are active,
- conditional on the sparsity pattern of `W`.

If any link in this chain is broken (zero weight, inactive ReLU, or missing linear connection), the gradient for that input vanishes.

This sparsity is not a numerical artifact — it is a structural property of the computation graph.

### Why are the figures necessary?

The figures serve two essential purposes.

**1. Revealing how upstream weights redistribute gradients**

The first figure shows how changing the upstream weights `v` **redistributes gradient magnitude** across different input components.

While the total sensitivity remains bounded, its allocation shifts smoothly as `v` changes.
This visually reinforces the idea that:

> vector–Jacobian products do not create new gradients — they reweight existing ones.

**2. Confirming a linear relationship between gradient ratios and weight ratios**

The second figure plots:

$$
\frac{\partial x_3}{\partial x_2}
\quad \text{vs.} \quad
\frac{v_3}{v_2}
$$
The near-perfect linear relationship confirms that:
- gradient ratios scale linearly with upstream weight ratios,
- the computation behaves exactly as predicted by the chain rule,
- ReLU’s gating only affects *when* gradients flow, not *how* they scale once active.

This provides strong empirical validation of the theory behind vector–Jacobian products.

## 🧭 Closing Remarks

This exercise demonstrates that in PyTorch:
- gradients of tensor-valued outputs are **not single objects**, but **directional sensitivities**,
- `backward(v)` computes how a *chosen direction in output space* propagates back to inputs,
- sparsity and structure in gradients arise naturally from the computation graph itself.

Together, the code and figures make vector–Jacobian products concrete, intuitive, and experimentally verifiable — exactly the conceptual bridge needed before moving on to optimisation in Part 2.